# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_data_path = "city_data.csv"
city_df = pd.read_csv(city_data_path)
city_df.describe()

,Unnamed: 0,Lat,Lon,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
count,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,550.000000,5.500000e+02
mean,274.500000,22.272488,16.069648,50.102673,74.503636,47.387273,8.133309,1.614398e+09
std,158.915596,32.197530,87.818273,31.189937,19.295681,40.633968,5.901106,2.211014e+02
min,0.000000,-54.800000,-179.166700,-31.790000,8.000000,0.000000,0.250000,1.614397e+09
25%,137.250000,-2.856050,-63.493125,35.010000,65.250000,1.000000,3.750000,1.614397e+09
50%,274.500000,26.997900,22.177350,59.000000,79.000000,40.000000,6.910000,1.614398e+09
75%,411.750000,49.158325,89.764375,75.390000,88.000000,90.000000,10.580000,1.614398e+09
max,549.000000,78.218600,179.316700,98.600000,100.000000,100.000000,37.980000,1.614398e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
coordinates = city_df[["Lat", "Lon"]].astype(float)
humidity =city_df['Humidity'].astype(float)

figure = gmaps.figure()

heat_map = gmaps.heatmap_layer(coordinates, weights=humidity,
                              dissipating=False, max_intensity=100,
                              point_radius=3)
figure.add_layer(heat_map)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_vacation_spot_df = city_df.loc[city_df['Max Temperature'] > 70]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Max Temperature'] < 80]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Wind Speed'] < 10]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Cloudiness'] == 0]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Humidity'] < 70]
ideal_vacation_spot_df = ideal_vacation_spot_df.loc[ideal_vacation_spot_df['Max Temperature'] < 78]

ideal_vacation_spot_df

,Unnamed: 0,City Name,Lat,Lon,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,Pedernales,18.0384,-71.7440,75.60,62,0,5.70,DO,1614397346
89,89,Busselton,-33.6500,115.3333,75.00,55,0,4.00,AU,1614397424
96,96,Lālganj,25.8667,85.1833,73.40,43,0,6.91,IN,1614397434
227,227,Tsabong,-26.0217,22.4010,72.10,62,0,7.54,BW,1614397613
234,234,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,75.99,53,0,6.24,AR,1614397621
281,281,Bubaque,11.2833,-15.8333,77.83,66,0,4.79,GW,1614397684
344,344,Oussouye,12.4850,-16.5469,73.40,60,0,6.91,SN,1614397767
410,410,Hedaru,-4.5000,37.9000,71.47,66,0,1.97,TZ,1614397854
431,431,Gouré,13.9835,10.2704,71.01,17,0,7.00,NE,1614397882


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_vacation_spot_df.copy()

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_coordinates = []
target_search = "hotel"
target_radius = 5000

hotel_name = []
hotel_lat = []
hotel_lng = []
hotel_city = []
hotel_country = []

city_list = ideal_vacation_spot_df['City Name'].values.tolist()
#latitude = ideal_vacation_spot_df['Lat'].values.tolist()
#longitude = ideal_vacation_spot_df['Lon'].values.tolist()



In [7]:
for index, row in ideal_vacation_spot_df.iterrows():
    
    try:
        lat = row['Lat']
        lon = row['Lon']

        params = {
            "location": f"{lat},{lon}",
            "keyword": target_search,
            "radius": target_radius,
            "key": g_key
            }
        response = requests.get(base_url, params=params).json()

        hotel_name.append(response['results'][0]["name"])
        hotel_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(response["results"][0]["geometry"]["location"]["lng"])
        hotel_city.append(row['City Name'])
        hotel_country.append(row['Country'])

        #ideal_vacation_spot_df["Hotel Name"] = places_data["results"][0]["name"]
        #hotel_df.loc[hotel_df['City Name'] == city, "Hotel Name"] = places_data["results"][0]["name"]

        time.sleep(2)

        print(hotel_name)
    
    except:
        print("Error with api call, moving on")
#print(ideal_vacation_spot_df["Hotel Name"])

['Hotel Alba Coral']
['Hotel Alba Coral', 'The Sebel Busselton']
Error with api call, moving on
['Hotel Alba Coral', 'The Sebel Busselton', 'Dikukama Hotel']
['Hotel Alba Coral', 'The Sebel Busselton', 'Dikukama Hotel', 'Hotel Gualok']
['Hotel Alba Coral', 'The Sebel Busselton', 'Dikukama Hotel', 'Hotel Gualok', 'Le Mango Lodge']
Error with api call, moving on
['Hotel Alba Coral', 'The Sebel Busselton', 'Dikukama Hotel', 'Hotel Gualok', 'Le Mango Lodge', 'Hedaru Palace Hotel']
Error with api call, moving on


In [8]:
hotel_data = {
    'Hotel Name' : hotel_name,
    'Lat' : hotel_lat,
    'Lng' : hotel_lng,
    'City' : hotel_city,
    'Country' : hotel_country
}

hotel_df = pd.DataFrame(hotel_data)

In [9]:
hotel_df.head()

,Hotel Name,Lat,Lng,City,Country
0,Hotel Alba Coral,18.037498,-71.739019,Pedernales,DO
1,The Sebel Busselton,-33.659693,115.278613,Busselton,AU
2,Dikukama Hotel,-26.012553,22.409274,Tsabong,BW
3,Hotel Gualok,-26.781256,-60.437947,Presidencia Roque Sáenz Peña,AR
4,Le Mango Lodge,11.303124,-15.838644,Bubaque,GW


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
figure.add_layer(markers)
# Display figure
figure

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>